<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fit-by-rank-difference" data-toc-modified-id="Fit-by-rank-difference-1">Fit by rank difference</a></span></li><li><span><a href="#local-models" data-toc-modified-id="local-models-2">local models</a></span><ul class="toc-item"><li><span><a href="#new" data-toc-modified-id="new-2.1">new</a></span></li></ul></li><li><span><a href="#model-coefficients-by-rank-difference" data-toc-modified-id="model-coefficients-by-rank-difference-3">model coefficients by rank difference</a></span></li></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Fit-by-rank-difference" data-toc-modified-id="Fit-by-rank-difference-1">Fit by rank difference</a></span></li><li><span><a href="#local-models" data-toc-modified-id="local-models-2">local models</a></span><ul class="toc-item"><li><span><a href="#new" data-toc-modified-id="new-2.1">new</a></span></li></ul></li><li><span><a href="#model-coefficients-by-rank-difference" data-toc-modified-id="model-coefficients-by-rank-difference-3">model coefficients by rank difference</a></span></li></ul></div>

In [ ]:
%run -i experiments_common.py

In [ ]:
spark

In [ ]:
traindata = pd.read_json('../pref_pair_sets/2019-05-01_uniform-samples_21-axioms.jsonl', lines=True)

In [ ]:
g = traindata.groupby(['query', 'system'])
print(len(traindata), 'pairs total')
print(g.concordant.count().head(1).values[0], 'pairs per query/system')
oneq = traindata.query('query == "301" & system == "DRMM"')
oneq[['rankdiff']].plot.hist(bins=100, title='Distribution of rank distance in training set')
plt.gcf().set_facecolor('white')

In [ ]:
print(*[f'{i:2d}. {j}' for i,j in enumerate(
    [x.split('ax_')[1] for x in traindata.columns if 'ax_' in x], 1)], sep='\n')

In [ ]:
fn = !readlink -f ../pref_pair_sets/2019-05-01_uniform-samples_21-axioms.jsonl
fn = fn[-1]
fn = f'file://{fn}'
fn

#fn = 'axiomatic-explainability-train-1'

In [ ]:
!curl -I $fn

In [ ]:
df = spark.read.json(fn)

In [ ]:
df.printSchema()

In [ ]:
col_features = [x for x in df.columns if x.startswith('ax_')]
axioms = [x.replace('ax_', '') for x in col_features]
systems = list(sorted(x['system'] for x in df.select('system').distinct().collect()))
col_target = 'concordant'

In [ ]:
tmp = df[(df.query == '356')&(df.system == 'DRMM')].toPandas()

# Fit by rank difference

In [ ]:
output_schema = T.StructType([
    T.StructField('query', T.StringType(), False), T.StructField('system', T.StringType(), False),    
    T.StructField('rankdiff', T.IntegerType(), False),
    T.StructField('prediction', T.ShortType(), False),
    T.StructField('truth', T.ShortType(), False),
    T.StructField('correct', T.ShortType(), False)
]
)

group_on = ['query', 'system']



@pandas_udf(output_schema, PandasUDFType.GROUPED_MAP)
def fit_by_rdiff(pdf):
    ix = np.array(pdf.index)
    np.random.shuffle(ix)
    ntest = len(ix)//3
    testidx = ix[:ntest]
    trainidx = ix[ntest:]
    
    train = pdf.loc[trainidx]
    test = pdf.loc[testidx]
    
    
    penalty = 'l1'
    C = 1
    
    xtr, ytr = train[col_features].values, train[col_target]

    cls = sklearn.ensemble.RandomForestClassifier()
    #cls = LogisticRegression(solver='saga', penalty=penalty, C=C)
    model = cls.fit(xtr, ytr)
    
    test['prediction'] = model.predict(test[col_features].values)
    test['truth'] = test.concordant
    test['correct'] = (test.prediction == test.truth).astype(int)
        
    group_key = list(pdf[group_on].iloc[0])
    columns = group_on + ['rankdiff', 'prediction', 'truth', 'correct']

    return test[columns]

d_fit_rdiff = df.groupBy(['query', 'system']).apply(fit_by_rdiff)
dtest = d_fit_rdiff.toPandas()

In [ ]:
g = d_fit_rdiff.groupby(['rankdiff'])
fit_rdiff_stats = g.agg(sqlf.mean('correct'), sqlf.stddev_samp('correct')).toPandas()

In [ ]:
fit_rdiff_stats.plot.scatter('rankdiff', 'avg(correct)')

# local models

In [ ]:

tmp = df

group_on = ['query', 'system']


output_schema = T.StructType([
    T.StructField('query', T.StringType(), False), T.StructField('system', T.StringType(), False),    
] + [
    T.StructField(a, T.FloatType(), False) for a in col_features
] + [
    T.StructField('penalty', T.StringType(), False),
    T.StructField('C', T.FloatType(), False),
    T.StructField('cv_accuracy_mean', T.FloatType(), False),
    T.StructField('cv_accuracy_std', T.FloatType(), False),
]
)

@pandas_udf(output_schema, PandasUDFType.GROUPED_MAP)
def train_partial_model(pdf):
    xs = pdf[col_features].values
    ys = pdf[col_target]
    group_key = list(pdf[group_on].iloc[0])
    columns = group_on + col_features + ['penalty', 'C', 'cv_accuracy_mean', 'cv_accuracy_std']
    out = pd.DataFrame(columns=columns)
    
    for penalty in ['l1']:
        for C in [0.01, 0.1, 0.5, 1]:
    
            cls = LogisticRegression(solver='saga', penalty=penalty, C=C)
            scores = cross_val_score(cls, xs, ys, cv=10)
            model = cls.fit(xs, ys)


            run = pd.DataFrame([
                group_key + list(model.coef_.flatten()) + [penalty, C, scores.mean(), scores.std()] ],
                columns=columns
            )
            out = out.append(run, ignore_index=True)
    
    return out
    


In [ ]:
%%time
localmodels = tmp.repartition(900).groupBy(['query', 'system']).apply(train_partial_model).toPandas()

In [ ]:
localmodels.head()

In [ ]:
traindata.sample(n=int(3e5))

In [ ]:
%%time
# sample = (df
#           #.filter('query == "427" and system == "DRMM"')
#           .toPandas())

sample = traindata.sample(n=int(3e5))

xs = sample[col_features].values
ys = sample[col_target].values

gb = sklearn.ensemble.GradientBoostingClassifier(verbose=0,
    n_estimators=100)
rf = sklearn.ensemble.RandomForestClassifier(verbose=0,
    n_estimators=100, max_depth=10)
lr = lambda C: LogisticRegression(solver='saga', penalty='l1', C=C)

scores = lambda cls: cross_val_score(cls, xs, ys, cv=5)

scores_df = pd.DataFrame(
    {'rf': scores(rf), 'gb': scores(gb), 'lr_.1': scores(lr(.1)),
     'lr_.01': scores(lr(.01)), 'lr_1': scores(lr(1))})
scores_df.describe()

In [ ]:
#spark.stop()


In [ ]:
localmodels.head()

In [ ]:
localmodels[['cv_accuracy_mean', 'cv_accuracy_std']].describe().T

In [ ]:
localmodels[['cv_accuracy_mean', 'cv_accuracy_std']].describe().T

In [ ]:
localmodels.groupby(['system', 'penalty', 'C']).cv_accuracy_mean.describe()

In [ ]:
systems = sorted(localmodels.system.unique())

In [ ]:
tmp = localmodels.copy()
tmp['reg'] = tmp.C.map(lambda x: f'l1, C={x:.2f}')
acc_by_query = (- tmp.groupby('query').cv_accuracy_mean.median()).rank()
tmp['q_acc'] = tmp['query'].map(lambda q: f'{int(acc_by_query[q])}.{q}')


In [ ]:

sns.barplot(
    data=tmp, x='q_acc', y='cv_accuracy_mean', hue='system', 
    hue_order=systems,
    ci='sd', capsize=.1)
# plt.gca().set_xticklabels(tmp['query'])
plt.ylim(0, 1)
plt.grid()
plt.title('Explanation accuracy by query (Error bars = sd across different regularization schemes)')
plt.gca().set_xticklabels(
    [tl._text.split('.')[-1] for tl in plt.gca().get_xticklabels()]
)
plt.gcf().set_size_inches((40,6))
plt.gcf().set_facecolor('white')

In [ ]:
# heatmap -- query vs axiom? / per system

coeffs = pd.melt(
    tmp, id_vars=['system', 'query', 'reg'], 
    value_vars=col_features, var_name='axiom', value_name='coeff')
coeffs['axiom'] = coeffs.axiom.map(lambda n: n.replace('ax_', ''))

def draw_heatmap(data, **kwargs):
    #print(data)
    data = data.pivot('query', 'axiom', 'coeff')[axioms]
    annot = data.round(2)
    sns.heatmap(data, cmap=sns.cm.vlag_r, vmin=-1, vmax=1, annot=False, cbar=False)
    plt.yticks(rotation=0)

with sns.plotting_context(font_scale=.75):
    g = sns.FacetGrid(coeffs, row='system', col='reg', sharex=True, sharey=True)
    g = g.map_dataframe(draw_heatmap)

plt.gcf().set_facecolor('white')
plt.gcf().set_size_inches((20, 60))
plt.gcf().tight_layout()

In [ ]:
# boxplots of coeff distribution

g = sns.catplot(data=coeffs, kind='boxen', x='axiom', y='coeff', hue='system', row='reg', hue_order=systems)

def adjust(a):
    a.grid()
    a.axhline()
    
[adjust(a) for a in g.axes.flatten()]

plt.suptitle('Distribution of axiom coefficients in query-local models', y=1.05)
plt.gcf().set_facecolor('white')
plt.gcf().set_size_inches((20, 12))

## new

In [ ]:
import pylab
import scipy.spatial.distance as sdist
import scipy.cluster.hierarchy as sch


ddata = localmodels.query("system == 'DRMM' & C == 0.50")

dmat = sdist.pdist(ddata[col_features], metric=sdist.cosine)
dmat = sdist.squareform(dmat)
#idx = localmodels[['system', 'query']]
#sns.heatmap(dmat)

fig = pylab.figure()
axdendro = fig.add_axes([0.09,0.1,0.2,0.8])
Y = sch.linkage(dmat, method='centroid')
Z = sch.dendrogram(Y, orientation='left')
axdendro.set_xticks([])
axdendro.set_yticks([])

axmatrix = fig.add_axes([0.3,0.1,0.6,0.8])
index = Z['leaves']
D = np.array(dmat)
D = D[index,:]
D = D[:,index]
im = axmatrix.matshow(D, aspect='auto', origin='lower')
axmatrix.set_xticklabels(ddata.iloc[index]['query'].values)
axmatrix.set_yticks([])

axcolor = fig.add_axes([0.91,0.1,0.02,0.8])
pylab.colorbar(im, cax=axcolor)

fig.set_size_inches((10,8))

In [ ]:
from axiomatic.collection.trec import TrecRobustQueries

tq = TrecRobustQueries('../robust04/topics.robust04.301-450.601-700.txt')

In [ ]:
! ls ../robust04/topics*

In [ ]:
ddata.iloc[[1,2]]['query']

In [ ]:
indices = pd.DataFrame(np.dstack(np.unravel_index(dmat.ravel().argsort(0), (dmat.shape)))[0], columns=['x', 'y'])
topk = set([tuple(sorted(x)) for x in indices.query('x!=y').head(60).values])

for (i, j) in topk:
    i, j = ddata.iloc[[i, j]]['query'].values
    #i, j = '%03d' %i, '%03d' % j
    print(i, j,  '|', tq[i].text, '/', tq[j].text)


In [ ]:
tq._q

In [ ]:
ddata.iloc[index]['query'].values

In [ ]:
localmodels

In [ ]:
# system-wise pca by coeff
from sklearn.decomposition import PCA
def draw_pca(data, **kwargs):
    pca = PCA(n_components=2)
    xs = data[col_features]
    pca.fit(xs)
    xs2 = pca.transform(xs)
    plt.scatter(xs2[:, 0], xs2[:, 1])

g = sns.FacetGrid(tmp, row='system', col='reg', sharex=True, sharey=True)
g = g.map_dataframe(draw_pca)

plt.suptitle('Independently fitted PCAs of query-local model coefficients', y=1.03)
plt.gcf().set_facecolor('white')

In [ ]:
tmp[tmp.cv_accuracy_mean > .7].plot

In [ ]:
tmp[tmp.cv_accuracy_mean < .6].plot.scatter('ax_PROX3', 'ax_REG')
plt.gcf().set_facecolor('white')

In [ ]:
sys.path.append('../../src/')
from axiomatic.collection.trec import TrecRobustQueries
queries = TrecRobustQueries('../robust04/topics.robust04.301-450.601-700.txt')
#list(queries._q.values())

In [ ]:
queries[401]

In [ ]:
# query clustering by coeff



In [ ]:
df1 = localmodels.copy()
df1 = df1[df1.system=='DRMM']
df1.drop(columns=['ax_AND', 'ax_M_AND'], inplace=True)
cols = [c for c in df1.columns if 'ax_' in c]
df1.head()
pd.plotting.radviz(df1[cols + ['system']], 'system')
plt.gcf().set_size_inches((10,10))

In [ ]:

def run(system, penalty, C, sample_weight=None):
    res = dict(system=system, penalty=penalty, C=C, sample_weight=sample_weight)

    tmp = df[(df['system'] == system)]
    cls = LogisticRegression(penalty=penalty, C=C, solver='saga')
    xs = tmp[col_features].values
    ys = tmp[col_target]

    model_key = (system, penalty, C, sample_weight)
    
    if sample_weight == 'relative_rankdiff':
        sample_weight = tmp.rankdiff / tmp.rankdiff.max()
    elif sample_weight == 'scorediff':
        sample_weight = tmp.scorediff
    
    model = cls.fit(xs, ys, sample_weight=sample_weight)
    
    models[model_key] = model
    accuracy = model.score(xs, ys, sample_weight=sample_weight)
    res['accuracy'] = accuracy

    coef = dict(zip(col_features, model.coef_.flatten()))
    res.update(coef)
    return res


# model coefficients by rank difference

In [5]:
from scipy import stats
bin_edges = stats.mstats.mquantiles(df.select(df.rankdiff).toPandas().values, np.linspace(0,1,7))
print(bin_edges)
plt.hist(dtest.rankdiff, bin_edges, color='w', edgecolor='k')
rdiff_bin_edges = bin_edges[1:]

@udf
def bin_label(rdiff):
    ix = rdiff_bin_edges.searchsorted(rdiff)
    if ix == 0:
        return f'rdiff<={rdiff_bin_edges[0]}'
    b = rdiff_bin_edges[max(0, ix-1):ix+1]
    return f'{b[0]}<rdiff<={b[1]}'


NameError: name 'df' is not defined

In [ ]:
df_binned = df.withColumn('binlabel', bin_label(df.rankdiff))

group_on = ['query', 'system', 'binlabel']

output_schema = T.StructType([
    T.StructField('query', T.StringType(), False), T.StructField('system', T.StringType(), False),    
    T.StructField('binlabel', T.StringType(), False)
] + [
    T.StructField(a, T.FloatType(), False) for a in col_features
] + [
    T.StructField('penalty', T.StringType(), False),
    T.StructField('C', T.FloatType(), False),
    T.StructField('cv_accuracy_mean', T.FloatType(), False),
    T.StructField('cv_accuracy_std', T.FloatType(), False),
]
)

@pandas_udf(output_schema, PandasUDFType.GROUPED_MAP)
def train_partial_model(pdf):
    xs = pdf[col_features].values
    ys = pdf[col_target]
    group_key = list(pdf[group_on].iloc[0])
    columns = group_on + col_features + ['penalty', 'C', 'cv_accuracy_mean', 'cv_accuracy_std']
    out = pd.DataFrame(columns=columns)
    
    for penalty in ['l1']:
        for C in [1]:
    
            cls = LogisticRegression(solver='saga', penalty=penalty, C=C)
            scores = cross_val_score(cls, xs, ys, cv=10)
            model = cls.fit(xs, ys)


            run = pd.DataFrame([
                group_key + list(model.coef_.flatten()) + [penalty, C, scores.mean(), scores.std()] ],
                columns=columns
            )
            out = out.append(run, ignore_index=True)
    
    return out

localmodels_binned = df_binned.groupby(group_on).apply(train_partial_model).toPandas()

In [ ]:
df_binned.head()

In [ ]:
df_binned.filter('query = "356"').filter('system = "DRMM"').groupby('binlabel').count().collect()

In [ ]:
binl = list(localmodels_binned.binlabel.unique())
binl = sorted(binl, key=lambda x: float(x.split('=')[-1]))
localmodels_binned.binlabel = localmodels_binned.binlabel.astype('category')
localmodels_binned.binlabel.cat.reorder_categories(binl, inplace=True)

In [ ]:
localmodels_binned.groupby(['system', 'binlabel']).cv_accuracy_mean.describe()

In [ ]:
localmodels_binned.head()

In [ ]:
localmodels_binned.ax_PROX5.hist(bins=100)

In [ ]:
tmp = localmodels_binned.copy()
tmp['reg'] = tmp.C.map(lambda x: f'l1, C={x:.2f}')
acc_by_query = (- tmp.groupby('query').cv_accuracy_mean.median()).rank()
tmp['q_acc'] = tmp['query'].map(lambda q: f'{int(acc_by_query[q])}.{q}')

coeffs = pd.melt(
    tmp, id_vars=['system', 'query', 'reg', 'binlabel'], 
    value_vars=col_features, var_name='axiom', value_name='coeff')
coeffs['axiom'] = coeffs.axiom.map(lambda n: n.replace('ax_', ''))


g = sns.catplot(data=coeffs, kind='boxen', x='axiom', y='coeff', hue='system', row='binlabel', 
                hue_order=systems, row_order=binl)

def adjust(a):
    a.grid()
    a.axhline()
    
[adjust(a) for a in g.axes.flatten()]

plt.suptitle('Distribution of axiom coefficients in query-local models', y=1.05)
plt.gcf().set_facecolor('white')
plt.gcf().set_size_inches((20, 42))